<a href="https://colab.research.google.com/github/prabhu44448/Face_News_Detection/blob/main/FakeNewsDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import json
import csv
import random

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers

import pprint
import tensorflow.compat.v1 as tf
from tensorflow.python.framework import ops
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
tf.disable_eager_execution()

# Reading the data
data = pd.read_csv("news.csv")

data2 = pd.read_csv("news2.csv")


data.head()


,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [ ]:
# encoding the labels
le = preprocessing.LabelEncoder()
le.fit(data['label'])
data['label'] = le.transform(data['label'])


In [ ]:
embedding_dim = 50
max_length = 54
trunc_type = 'post'
padding_type = 'post'
oov_tok = "<OOV>"
training_size = 6300
test_portion = .1


In [ ]:
title = []
text = []
labels = []
for x in range(training_size):
	title.append(data['title'][x])
	text.append(data['text'][x])
	labels.append(data['label'][x])


In [ ]:
tokenizer1 = Tokenizer()
tokenizer1.fit_on_texts(title)
word_index1 = tokenizer1.word_index
vocab_size1 = len(word_index1)
sequences1 = tokenizer1.texts_to_sequences(title)
padded1 = pad_sequences(
	sequences1, padding=padding_type, truncating=trunc_type)
split = int(test_portion * training_size)
training_sequences1 = padded1[split:training_size]
test_sequences1 = padded1[0:split]
test_labels = labels[0:split]
training_labels = labels[split:training_size]


In [ ]:
embeddings_index = {}
with open('glove.6B.50d.txt') as f:
	for line in f:
		values = line.split()
		word = values[0]
		coefs = np.asarray(values[1:], dtype='float32')
		embeddings_index[word] = coefs

# Generating embeddings
embeddings_matrix = np.zeros((vocab_size1+1, embedding_dim))
for word, i in word_index1.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embeddings_matrix[i] = embedding_vector
print(embeddings_matrix.shape)
print(embedding_dim)

(11687, 50)
50


In [ ]:
model = tf.keras.Sequential([
	tf.keras.layers.Embedding(vocab_size1+1, embedding_dim,
							input_length=max_length, weights=[
								embeddings_matrix],
							trainable=False),
	tf.keras.layers.Dropout(0.2),
	tf.keras.layers.Conv1D(64, 5, activation='relu'),
	tf.keras.layers.MaxPooling1D(pool_size=4),
	tf.keras.layers.LSTM(64),
	tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',
			optimizer='adam', metrics=['accuracy'])
model.summary()


Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_20 (Embedding)    (None, 54, 50)            584350    
                                                                 
 dropout_20 (Dropout)        (None, 54, 50)            0         
                                                                 
 conv1d_20 (Conv1D)          (None, 50, 64)            16064     
                                                                 
 max_pooling1d_20 (MaxPoolin  (None, 12, 64)           0         
 g1D)                                                            
                                                                 
 lstm_20 (LSTM)              (None, 64)                33024     
                                                                 
 dense_20 (Dense)            (None, 1)                 65        
                                                     

In [ ]:
num_epochs = 250

training_padded = np.array(training_sequences1)
training_labels = np.array(training_labels)
testing_padded = np.array(test_sequences1)
testing_labels = np.array(test_labels)
print(training_sequences1.shape)
history = model.fit(training_padded, training_labels,
					epochs=num_epochs,
					validation_data=(testing_padded,
									testing_labels),
					verbose=2)


(5670, 54)
Train on 5670 samples, validate on 630 samples
Epoch 1/250


/usr/local/lib/python3.9/dist-packages/keras/engine/training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


5670/5670 - 8s - loss: 0.6209 - accuracy: 0.6489 - val_loss: 0.5380 - val_accuracy: 0.7190 - 8s/epoch - 1ms/sample
Epoch 2/250
5670/5670 - 6s - loss: 0.5536 - accuracy: 0.7178 - val_loss: 0.5063 - val_accuracy: 0.7397 - 6s/epoch - 1ms/sample
Epoch 3/250
5670/5670 - 5s - loss: 0.5033 - accuracy: 0.7494 - val_loss: 0.4748 - val_accuracy: 0.7683 - 5s/epoch - 852us/sample
Epoch 4/250
5670/5670 - 6s - loss: 0.4606 - accuracy: 0.7776 - val_loss: 0.4678 - val_accuracy: 0.7794 - 6s/epoch - 1ms/sample
Epoch 5/250
5670/5670 - 5s - loss: 0.4320 - accuracy: 0.7922 - val_loss: 0.4619 - val_accuracy: 0.7841 - 5s/epoch - 881us/sample
Epoch 6/250
5670/5670 - 5s - loss: 0.4004 - accuracy: 0.8194 - val_loss: 0.4555 - val_accuracy: 0.8016 - 5s/epoch - 888us/sample
Epoch 7/250
5670/5670 - 6s - loss: 0.3645 - accuracy: 0.8386 - val_loss: 0.4435 - val_accuracy: 0.8000 - 6s/epoch - 1ms/sample
Epoch 8/250
5670/5670 - 5s - loss: 0.3445 - accuracy: 0.8478 - val_loss: 0.4572 - val_accuracy: 0.7857 - 5s/epoch - 8

In [ ]:
# sample text to check if fake or not
X = "Kerry to go to Paris in gesture of sympathy"

# detection
sequences = tokenizer1.texts_to_sequences([X])[0]
sequences = pad_sequences([sequences], maxlen=54,
						padding=padding_type,
						truncating=trunc_type)
print(sequences)
a=model.predict(sequences,verbose=0)
print(a[0][0])
s=''
if a<0.2:
	print("fake")
	s='FAKE'

elif a<0.4:
	print("Hate")
	s='HATE'
elif a<0.6:
	s='OFFENSIVE'
	print("Offensive")
elif a<0.8:
	print("Hostile")
	s='HOSTILE'
elif a<1:
	s='REAL'
	print("News is Real")



[[ 558    2  197    2  225    3 3451    4 3452    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0]]
0.9871332
News is Real


In [ ]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))
from flask import Flask,render_template,request
import requests
a=a[0][0]*100
app = Flask(__name__,template_folder='/content/')
print(a)
@app.route('/',methods=['GET', 'POST'])
def hello():
    # print("ggg")
    # name = request.form['text']
    # print(name)
    #x='News is '+s+' with '+a+'% Accuracy'
    # print(a)
    # if(a>0.5):
    #   return "This news is True"
    # else:
    #   return "This news is false"
    return render_template('xx.html',s=s,a=a)

if __name__ == '__main__':
    app.run(port=5000)


https://16221q6rp9u-496ff2e9c6d22116-5000-colab.googleusercontent.com/
98.71332049369812
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [24/Apr/2023 01:36:20] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Apr/2023 01:36:21] "GET /favicon.ico HTTP/1.1" 404 -
